# Data analysis

This notebooks is used to analyse bunch electrons in a Plasma Wakefield simulation.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import openpmd_api as io
from scipy import constants
from scipy.optimize import curve_fit
import scipy.integrate as integrate

from matplotlib.colors import LogNorm, SymLogNorm, Normalize
from matplotlib.colors import hsv_to_rgb
from scipy import stats

In [ ]:
#path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/test/simOutput/openPMD/simData_beta_%T.bp"
path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/particles_250MeV/035_particles_bachelor1/simOutput/openPMD/simData_beta_%T.bp"
path_field="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/downramp_250MeV/030_downramp_bachelor1/simOutput/openPMD/simData_%T.bp"

## Definitions

Define different functions for data fitting.

In [ ]:
def window(t):
    """Calculates width of viewing window."""
    y_cells = 2048
    z_cells = 1024      # should be same as x_cells
    y_gpus = 8
    cellwidth = 0.5 * 0.1772e-6 * 1e6   # in micorometer, same for all directions
    Delta_t = 1.706e-16/1.28631
    windowwidth = y_cells * (y_gpus-1)/y_gpus * cellwidth
    windowheight = z_cells * cellwidth

    y_min = Delta_t * constants.c * t * 1e6 + windowwidth * 0.52
    y_max = Delta_t * constants.c * t * 1e6 + windowwidth * 0.72
    z_min = -windowheight / 2
    z_max =  windowheight / 2
    
    return y_min, y_max, z_min, z_max

In [ ]:
def Gauss(x, a, b, c):
    y = a * np.exp(-(x-b)**2 / (2*c**2))
    return y

def Gauss_dens(x, a, b):
    y = 1/np.sqrt(2*np.pi * b**2) * np.exp(-(x-a)**2 / (2*b**2))
    return y

def Gauss2(x, a1, b1, c1, a2, b2, c2):
    y = a1 * np.exp(-(x-b1)**2 / (2*c1**2)) + a2 * np.exp(-(x-b2)**2 / (2*c2**2))
    return y

def Gauss3(x, a1, b1, c1):
    y = c1 * np.exp(-(x-a1)**2 / (2*b1**2))
    return y

def Lorentz(x, a, b):
    y = 2/np.pi * b / (4*(x - a)**2 + b)
    return y

def Max_Boltz_2D(x, a):
    y = x/a * np.exp(-x**2 / (2*a))
    return y

In [ ]:
def particlePMD(particle, pos=True, mom=True, probeE=False, probeB=False, Id=False):
    """ Returns unpacked particle data from openPMD.
    Always returns weights, optionally returns position, momentum and the exerted E- and B-fields on the particles.
    Position is returned in \mu m, the rest in SI-units.
    """
    
    if pos:
        x_off = particle["positionOffset"]["x"][:]
        x_off_unit = particle["positionOffset"]["x"].unit_SI
        y_off = particle["positionOffset"]["y"][:]
        y_off_unit = particle["positionOffset"]["y"].unit_SI
        z_off = particle["positionOffset"]["z"][:]
        z_off_unit = particle["positionOffset"]["z"].unit_SI

        x = particle["position"]["x"][:]
        x_unit = particle["position"]["x"].unit_SI
        y = particle["position"]["y"][:]
        y_unit = particle["position"]["y"].unit_SI
        z = particle["position"]["z"][:]
        z_unit = particle["position"]["z"].unit_SI

    if mom:
        p_x = particle["momentum"]["x"][:]
        p_x_unit = particle["momentum"]["x"].unit_SI
        p_y = particle["momentum"]["y"][:]
        p_y_unit = particle["momentum"]["y"].unit_SI
        p_z = particle["momentum"]["z"][:]
        p_z_unit = particle["momentum"]["z"].unit_SI

    if probeE:
        x_probeE = particle['probeE']['x'][:]
        x_probeE_unit = particle["probeE"]["x"].unit_SI 
        y_probeE = particle['probeE']['y'][:]
        y_probeE_unit = particle["probeE"]["y"].unit_SI
        z_probeE = particle['probeE']['z'][:]
        z_probeE_unit = particle["probeE"]["z"].unit_SI

    if probeB:
        x_probeB = particle['probeB']['x'][:]
        x_probeB_unit = particle["probeB"]["x"].unit_SI 
        y_probeB = particle['probeB']['y'][:]
        y_probeB_unit = particle["probeB"]["y"].unit_SI
        z_probeB = particle['probeB']['z'][:]
        z_probeB_unit = particle["probeB"]["z"].unit_SI

    if Id:
        particleId = particle["id"][io.Mesh_Record_Component.SCALAR][:]

    weight = particle["weighting"][io.Mesh_Record_Component.SCALAR][:]

    series.flush()

    pack = [weight]

    if pos:
        x_pos = (x * x_unit + (x_off-512) * x_off_unit) * 1e6
        y_pos = (y * y_unit +  y_off      * y_off_unit) * 1e6
        z_pos = (z * z_unit + (z_off-512) * z_off_unit) * 1e6
        pack = np.append(pack, [x_pos, y_pos, z_pos], axis=0)

    if mom:
        x_mom = p_x * p_x_unit
        y_mom = p_y * p_y_unit
        z_mom = p_z * p_z_unit
        pack = np.append(pack, [x_mom, y_mom, z_mom], axis=0)

    if probeE:
        x_probeE *= x_probeE_unit
        y_probeE *= y_probeE_unit
        z_probeE *= z_probeE_unit
        pack = np.append(pack, [x_probeE, y_probeE, z_probeE], axis=0)

    if probeB:
        x_probeB *= x_probeB_unit
        y_probeB *= y_probeB_unit
        z_probeB *= z_probeB_unit
        pack = np.append(pack, [x_probeB, y_probeB, z_probeB], axis=0)

    if Id:
        pack = np.append(pack, [particleId], axis=0)

    return pack

In [ ]:
def mom_to_E(x_mom, y_mom, z_mom, weight):
    # squared total momentum
    p2_total = np.float64(x_mom**2 + y_mom**2 + z_mom**2)
    # energy per macro particle in Joule
    E_J = (constants.c * np.sqrt((constants.m_e*weight)**2 * constants.c**2 + p2_total) - (constants.m_e*weight) * constants.c**2) * 1/weight
    # conversion to MeV
    E = E_J / constants.e / 1e6
    
    return E

In [ ]:
def binned_to_grid(bin_x, bin_y):
    """Transforms 2D bins into grid.
    The left borders of the bins are taken as gridpoints, the rightmost border is discarded.
    """
    x_width = bin_x[1] - bin_x[0]
    y_width = bin_y[1] - bin_y[0]
    x = bin_x[:-1] + x_width/2
    y = bin_y[:-1] + y_width/2
    
    return x, y

## Density fields

Plots the charge- and energy-density fields for b- and e-Particles

In [ ]:
# Test for density plots
series = io.Series(path_field, io.Access.read_only)


for i in range(56000, 90000+1, 2000):
    plt.figure(figsize=(16, 6), dpi=90)
    e_field_charge = series.iterations[i].meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    e_field_energy = series.iterations[i].meshes["e_all_energyDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    b_field_charge = series.iterations[i].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    b_field_energy = series.iterations[i].meshes["b_all_energyDensity"][io.Mesh_Record_Component.SCALAR][:,:,512]
    series.flush()
    
    print("e_field_charge: ", np.min(e_field_charge), np.max(e_field_charge))
    print("e_field_energy: ", np.min(e_field_energy), np.max(e_field_energy))
    print("b_field_charge: ", np.min(b_field_charge), np.max(b_field_charge))
    print("b_field_energy: ", np.min(b_field_energy), np.max(b_field_energy))
    
    plt.subplot(121)
    plt.imshow(-e_field_charge, cmap='hot_r', norm=LogNorm(vmin=1e-4, vmax=20))
    plt.colorbar(label="electrons")    
    plt.imshow(-b_field_charge, cmap='Greens', norm=LogNorm(vmin=1e-12, vmax=3))
    plt.colorbar(label="bunch") 
    plt.xlabel("y")
    plt.ylabel("z")
    plt.title("charge density, timestep = {}".format(i))
    
    plt.subplot(122)
    plt.imshow(e_field_energy, cmap='hot_r', norm=LogNorm(vmin=1e-10, vmax=1e2))
    plt.colorbar(label="electrons")    
    plt.imshow(b_field_energy, cmap='Greens', norm=LogNorm(vmin=1e-5, vmax=1e3))
    plt.colorbar(label="bunch")    
    plt.xlabel("y")
    plt.ylabel("z")
    plt.title("energy density, timestep = {}".format(i))
    
    plt.show()
    #plt.savefig(fname="temp", facecolor="white")

del series

# gibt es transparente Farbschemen?

## Phasespace

Plots the y-p_y (or other combinations) phase space.

In [ ]:
series = io.Series(path, io.Access.read_only)

for t in range(60000, 90000+1, 1000):
    particle = series.iterations[t].particles["b_all"]

    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom = particlePMD(particle)

    # phase histogram
    plt.figure(figsize=(8, 6), dpi=90)
    plt.hist2d(y_pos, y_mom, bins=200, weights=weight, cmap="hot_r")
    plt.colorbar()
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$p_y\, \mathrm{[Ns]}$")
    plt.title("y-p_y-phase space, timestep = {}".format(t))
    plt.show()
    #plt.savefig('temp.png', facecolor='w')

del series

## Energy-Divergence Histograms

In [ ]:
time = range(24000, 120000+1, 2000)
E_bin_num = 100                    # Bin number for energy histogram
peak_energy = 244.5             # peak energy for masks
div_max = 60                    # maximal range of viewed divergence

# energy-divergence histogram
range_E = [0, 400]
range_div = [-div_max, div_max]
bins1 = [1000, 201]
 
dE = (range_E[1] - range_E[0]) / bins1[0]
ddiv = (range_div[1] - range_div[0]) / bins1[1]
dQ = constants.e * 1e12

# divergence map
bins2 = 201

# energy fit
mask = (pos_E>175)

# divergence fit
# calculates index of given energy when changing the xbins or xrange
ind_Estart = int(240/dE)
ind_Eend = int(250/dE)
p1 = [10, 0, 3]


E_peak = np.zeros(len(time))
E_peak_stat = np.zeros(len(time))
E_peak_sys = np.zeros(len(time))
E_max = np.zeros(len(time))
E_min = np.zeros(len(time))
E_mean = np.zeros(len(time))
E_std = np.zeros(len(time))
E_hist = np.zeros((len(time), E_bin_num))    # stores energy histogram for every timestep

# return maximal energy over all timesteps
series = io.Series(path, io.Access.read_only)
particle_end = series.iterations[time[-1]].particles["b_all"]
weight_end, x_mom_end, y_mom_end, z_mom_end, particleId_end = particlePMD(particle_end, pos=False, Id=True)
E_end = mom_to_E(x_mom_end, y_mom_end, z_mom_end, weight_end)
max_Energy = np.max(E_end) + 5

for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    y_min, y_max, z_min, z_max = window(t)

    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom, particleId = particlePMD(particle, Id=True)

    E = mom_to_E(x_mom, y_mom, z_mom, weight)

    E_max[i] = np.max(E)
    E_min[i] = np.min(E)
    E_mean[i] = np.mean(E)
    E_std[i] = np.std(E)
    E_hist[i][:] = np.histogram(E, weights=weight, bins=E_bin_num, range=(0, max_Energy))[0] * dQ / dE

    # divergence (or whatever it's actually called)
    div = np.arctan2(z_mom, y_mom)*1e3     # should be x_mom
    
    
    # energy-divergence histogram
    hist, edges_E, edges_div= np.histogram2d(E, div, weights=weight, bins=bins1, range=[range_E, range_div])
    hist *= dQ / dE / ddiv


    # divergence map
    dy = (y_max - y_min) / bins2
    dz = (z_max - z_min) / bins2
    statistic_div = stats.binned_statistic_2d(y_pos, z_pos, (np.abs(div)), statistic='mean', bins=bins2, range=[[y_min, y_max], [z_min, z_max]])[0]
    statistic_div *= 1 / dy / dz


    # energy fit
    pos_E = edges_E[:-1] + dE/2
    height_E = np.sum(hist, axis=1)*ddiv

    # fitting to Gauss curve
    p0 = [15, 245, 6, 5, 205, 18]
    parameters, covariance = curve_fit(Gauss2, pos_E[mask], height_E[mask], p0)
    fit_a1, fit_b1, fit_c1, fit_a2, fit_b2, fit_c2 = parameters
    fit_E = Gauss2(pos_E, fit_a1, fit_b1, fit_c1, fit_a2, fit_b2, fit_c2)
    
    print("Energy fit:\n  Peak 1:     Height: {:.2f}, Mean: {:.2f} MeV, Std. Dev.: {:.2f} MeV\n  Peak 2:     Height: {:.2f}, Mean: {:.2f} MeV, Std. Dev.: {:.2f} MeV".format(fit_a1, fit_b1, fit_c1, fit_a2, fit_b2, fit_c2))
    E_peak[i] = fit_b1
    E_peak_stat[i] = np.sqrt(covariance[1,1])
    E_peak_sys[i] = dE


    # divergence fit
    pos_div = edges_div[:-1] + ddiv/2
    height_div = np.sum(hist[ind_Estart:ind_Eend,:], axis=0)*dE

    # fitting to Gauss curve
    parameters, covariance = curve_fit(Gauss, pos_div, height_div, p1)
    fit_a, fit_b, fit_c = parameters
    fit_y = Gauss(pos_div, fit_a, fit_b, fit_c)
    print("Divergence fit:\n  Height: {:.2f}, Mean: {:.2f} mrad, Std. Dev.: {:.2f} mrad".format(fit_a, fit_b, fit_c))

    plot = False
    if plot:
        plt.figure(figsize=(16, 10), dpi=70)

        # energy-divergence plot
        plt.subplot(221)
        plt.imshow(hist.T, origin="lower", interpolation='none', aspect='auto', cmap="hot_r", extent=(range_E[0], range_E[1], range_div[0], range_div[1]), norm=LogNorm(vmin=5e-3))
        plt.colorbar(label="$dQ/dE/d\\theta \, \mathrm{[pC/Mev/mrad]}$")
        plt.xlabel("energy [MeV]")
        plt.ylabel("divergence [mrad]")
        plt.title("t: {:1d}".format(t))

        plt.subplot(222)
        plt.imshow(statistic_div.T, cmap='magma_r', extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label='$d\\theta/dy/dz \, \mathrm{[mrad/\mu m^2]}$')
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("absolute divergence, timstep: {:1d}".format(t))

        plt.subplot(223)
        plt.bar(pos_E, height_E, label="data")
        plt.plot(pos_E[mask], fit_E[mask], label="fit", c='orange')
        plt.xlabel("energy [MeV]")
        plt.ylabel("$dQ/dE \, \mathrm{[pC/Mev]}$")
        plt.xlim([0,300])
        plt.legend()
        plt.title("Energy count, t: {:1d}, |div| < {:.1f} mrad".format(t, div_max))

        plt.subplot(224)
        plt.bar(pos_div, height_div, label='data',width=0.5)
        plt.plot(pos_div, fit_y, label='fit', c='orange')
        plt.xlabel("divergence [mrad]")
        plt.ylabel("$dQ/d\\theta \, \mathrm{[pC/mrad]}$")
        plt.xlim([-40,40])
        plt.title("250 MeV > energy > 240 MeV, t: {:1d}".format(t))
        plt.legend()

        #plt.savefig(fname="temp", facecolor="white")
        plt.show()
    else:
        print("timestep: ", t, "\n")

del series

In [ ]:
plt.figure(figsize=(16, 4), dpi=90)

plt.subplot(121)
plt.imshow(E_hist.T, origin='lower', extent=(time[0]-1e3, time[-1]+1e3, 0, max_Energy), aspect='auto', interpolation='none')
plt.plot(time, E_max, marker='.', label='E_max')
plt.errorbar(time, E_mean, yerr=E_std, marker='.', label='E_mean', capsize=3)
plt.plot(time, E_peak, marker='.', label='E_peak', c='red')
plt.plot(time, E_min, marker='.', label='E_min')
plt.colorbar(label='$Q/E \, \mathrm{[pC/MeV]}$')     # korrekte Einheit?
plt.xlabel("timestep")
plt.ylabel("energy [MeV]")
plt.title("energy over time")
plt.legend()

plt.subplot(122)
plt.errorbar(time, E_peak, yerr=E_peak_stat, marker='.', capsize=3, label="data")
plt.plot(time, E_peak+E_peak_sys, c='orange', label="sys. uncertainty")
plt.plot(time, E_peak-E_peak_sys, c='orange')
plt.hlines(np.mean(E_peak), time[0], time[-1], color='black')
plt.xlabel("timestep")
plt.ylabel("energy [MeV]")
plt.title("peak energy over time")
plt.legend()

plt.show()
#plt.savefig(fname="temp", facecolor="white", dpi=100)

## Peak Energy movement

In [ ]:
time = range(30000, 50000+1, 2000)
peak_energy = 244.5             # peak energy for masks

# start reading openPMD data
series = io.Series(path, io.Access.read_only)

# return id's of particles, which have peak energy at the last timestep
particle = series.iterations[time[-1]].particles["b_all"]
weight_end, x_mom_end, y_mom_end, z_mom_end, particleId_end = particlePMD(particle, pos=False, Id=True)
E_end = mom_to_E(x_mom_end, y_mom_end, z_mom_end, weight_end)
id_peak = particleId_end[(E_end>peak_energy-3)&(E_end<peak_energy+3)]

for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom, particleId = particlePMD(particle, Id=True)

    E = mom_to_E(x_mom, y_mom, z_mom, weight)
    
    # size of viewing window
    y_min, y_max, z_min, z_max = window(t)
    
    # shows only particles with E in given intervall
    E_mask = (E>peak_energy-2)&(E<peak_energy+2)
    id_mask = np.isin(particleId, id_peak)
    
    y_peak = y_pos[E_mask]
    z_peak = z_pos[E_mask]
    weight_peak = weight[E_mask]
    
    y_id_peak = y_pos[id_mask]
    z_id_peak = z_pos[id_mask]
    weight_id_peak = weight[id_mask]
    
    # plots
    plt.figure(figsize=(16, 5), dpi=70)
    
    # particle density plot where particles with peak density are highlighted
    plt.subplot(121)
    plt.hist2d(y_pos, z_pos, bins=200,  weights=weight, cmap='hot_r', norm=LogNorm(vmin=5e4, vmax=2e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (all particles)')
    plt.hist2d(y_peak, z_peak, bins=200,  weights=weight_peak, cmap='cool_r', norm=LogNorm(vmin=3e4, vmax=1e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (240 MeV < E < 250 MeV)')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))    
    
    # particle density plot where particles with peak density at step 90.000 are highlighted
    plt.subplot(122)
    plt.hist2d(y_pos, z_pos, bins=200,  weights=weight, cmap='hot_r', norm=LogNorm(vmin=5e4, vmax=2e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (all particles)')
    plt.hist2d(y_id_peak, z_id_peak, bins=200,  weights=weight_id_peak, cmap='cool_r', norm=LogNorm(vmin=3e4, vmax=2e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='density (240 MeV < E < 250 MeV on step 90.000)')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))   

    #plt.savefig(fname="temp", facecolor="white")
    plt.show()    

del series

## Electromagnetic Probe-Fields

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(12000, 20000+1, 2000)

for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    
    weight, x_pos, y_pos, z_pos, x_probeE, y_probeE, z_probeE, x_probeB, y_probeB, z_probeB = particlePMD(particle, mom=False, probeE=True, probeB=True)

    plt.figure(dpi=60, figsize=(24, 12))
    plt.subplot(231)
    plt.hist2d(y_pos, z_pos, bins=200,  weights=weight, cmap='hot_r', norm=LogNorm())
    plt.colorbar(label='particle density')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))
    
    statistic_E, bin_y_E, bin_z_E, rest_E = stats.binned_statistic_2d(y_pos, z_pos, (x_probeE*1e-9, y_probeE*1e-9, z_probeE*1e-9), statistic='mean', bins=60)
    y_E, z_E = binned_to_grid(bin_y_E, bin_z_E)
    abs_E = np.sqrt(statistic_E[0].T**2 + statistic_E[1].T**2 + statistic_E[2].T**2)
    
    plt.subplot(232)
    plt.streamplot(y_E, z_E, statistic_E[1].T, statistic_E[2].T, density=3, color=abs_E, cmap='magma_r')
    plt.colorbar(label='|probeE| [GV/m]')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z probeE, timstep: {:1d}".format(t))
    
    statistic_E, bin_x_E, bin_z_E, rest_E = stats.binned_statistic_2d(x_pos, z_pos, (x_probeE*1e-9, y_probeE*1e-9, z_probeE*1e-9), statistic='mean', bins=60)
    x_E, z_E = binned_to_grid(bin_x_E, bin_z_E)
    abs_E = np.sqrt(statistic_E[0].T**2 + statistic_E[1].T**2 + statistic_E[2].T**2)
    
    plt.subplot(233)
    plt.streamplot(x_E, z_E, statistic_E[0].T, statistic_E[2].T, density=3, color=abs_E, cmap='magma_r')
    plt.colorbar(label='|probeE| [GV/m]')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z probeE, timstep: {:1d}".format(t))
    
    probe_B_mask = (x_pos>0)
    statistic_B, bin_y_B, bin_z_B, rest_B = stats.binned_statistic_2d(y_pos[probe_B_mask], z_pos[probe_B_mask], (x_probeB[probe_B_mask], y_probeB[probe_B_mask], z_probeB[probe_B_mask]), statistic='mean', bins=60)
    y_B, z_B = binned_to_grid(bin_y_B, bin_z_B)
    abs_B = np.sqrt(statistic_B[0].T**2 + statistic_B[1].T**2 + statistic_B[2].T**2)
    
    plt.subplot(235)
    plt.streamplot(y_B, z_B, statistic_B[1].T, statistic_B[2].T, density=3, color=abs_B, cmap='magma_r')
    plt.colorbar(label='|probeB| [T]')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z probeB, timstep: {:1d}".format(t))
    
    statistic_B, bin_x_B, bin_z_B, rest_B = stats.binned_statistic_2d(x_pos, z_pos, (x_probeB, y_probeB, z_probeB), statistic='mean', bins=60)
    x_B, z_B = binned_to_grid(bin_x_B, bin_z_B)
    abs_B = np.sqrt(statistic_B[0].T**2 + statistic_B[1].T**2 + statistic_B[2].T**2)
                    
    plt.subplot(236)
    plt.streamplot(x_B, z_B, statistic_B[0].T, statistic_B[2].T, density=3, color=abs_B, cmap='magma_r')
    plt.colorbar(label='|probeB| [T]')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z probeB, timstep: {:1d}".format(t))
    
    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

## Lorentz-Force

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(20000, 20000+1, 2000)

for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    y_min, y_max, z_min, z_max = window(t)

    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom, x_probeE, y_probeE, z_probeE, x_probeB, y_probeB, z_probeB = particlePMD(particle, probeE=True, probeB=True)

    # converting momenta to velocity
    x_vel = np.sqrt((x_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))
    y_vel = np.sqrt((y_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))
    z_vel = np.sqrt((z_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))

    F_x_B = y_vel*z_probeB - z_vel*y_probeB
    F_y_B = z_vel*x_probeB - x_vel*z_probeB
    F_z_B = x_vel*y_probeB - y_vel*x_probeB
    
    F_x_L = -constants.e*weight * (x_probeE + F_x_B)
    F_y_L = -constants.e*weight * (y_probeE + F_y_B)
    F_z_L = -constants.e*weight * (z_probeE + F_z_B)
    print(np.sum(F_x_L), np.sum(F_y_L), np.sum(F_z_L))
    statistic, bin_y, bin_z, rest = stats.binned_statistic_2d(y_pos, z_pos, (F_x_L, F_y_L, F_z_L), statistic='mean', bins=60, range=[[y_min, y_max], [z_min, z_max]])
    y, z = binned_to_grid(bin_y, bin_z)
    col = np.sqrt(statistic[0].T**2 + statistic[1].T**2 + statistic[2].T**2)*1e3
    
    plt.figure(dpi=60, figsize=(24, 6))
    
    plt.subplot(131)
    plt.hist2d(y_pos, z_pos, bins=200,  weights=weight, cmap='hot_r', norm=LogNorm(), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='particle density')
    plt.streamplot(y, z, statistic[1].T, statistic[2].T, density=2, color=col, cmap='Greens')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))
    
    plt.subplot(132)
    plt.streamplot(y, z, statistic[1].T, statistic[2].T, density=3, color=col, cmap='magma_r')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z Lorentz-Force, timstep: {:1d}".format(t))
    
    statistic, bin_x, bin_z, rest = stats.binned_statistic_2d(x_pos, z_pos, (F_x_L, F_y_L, F_z_L), statistic='mean', bins=60, range=[[z_min, z_max], [z_min, z_max]])
    x, z = binned_to_grid(bin_x, bin_z)
    col = np.sqrt(statistic[0].T**2 + statistic[1].T**2 + statistic[2].T**2)*1e3
    
    plt.subplot(133)
    plt.streamplot(x, z, statistic[0].T, statistic[2].T, density=3, color=col, cmap='magma_r')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z Lorentz-Force, timstep: {:1d}".format(t))

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(12000, 90000+1, 2000)
z_mean = np.zeros(len(time))
y_mean = np.zeros(len(time))
p_z_mean = np.zeros(len(time))
F_y_mean = np.zeros(len(time))
F_y_peak = np.zeros(len(time))
F_z_mean = np.zeros(len(time))
F_z_std = np.zeros(len(time))

for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    
    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom, x_probeE, y_probeE, z_probeE, x_probeB, y_probeB, z_probeB = particlePMD(particle, probeE=True, probeB=True)
    x_vel = np.sqrt((x_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))
    y_vel = np.sqrt((y_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))
    z_vel = np.sqrt((z_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))

#    F_x_B = y_vel*z_probeB - z_vel*y_probeB
    F_y_B = z_vel*x_probeB - x_vel*z_probeB
    F_z_B = x_vel*y_probeB - y_vel*x_probeB
    
#    F_x_L = -constants.e*weight * (x_probeE + F_x_B)
    F_y_L = -constants.e*weight * (y_probeE + F_y_B)
    F_z_L = -constants.e*weight * (z_probeE + F_z_B)
    
    z_mean[i] = np.mean(z_pos)
    y_mean[i] = np.mean(y_pos)
    p_z_mean[i] = np.mean(z_mom)
    F_z_mean[i] = np.mean(F_z_L)
    F_z_std[i] = np.std(F_z_L) / np.sqrt(len(F_z_L))
    F_y_mean[i] = np.mean(F_y_L)

    E = mom_to_E(x_mom, y_mom, z_mom, weight)
    F_y_peak[i] = np.mean(F_y_L[(E>243)&(E<247)])
    
#    plt.hist2d(E, F_y_L*1e3, bins=100, norm=LogNorm(), cmap='cubehelix_r')
#    plt.xlabel("$E \, \mathrm{[MeV]}$")
#    plt.ylabel("$F_y \, \mathrm{[mN]}$")
#    plt.colorbar(label='count')
#    plt.title("timstep: {:1d}".format(t))    
#    plt.vlines(245, -1, 1, color='black')
    #plt.savefig(fname="temp", facecolor="white", dpi=200)
#    plt.show()
    #print(F_y_L[(E>243)&(E<247)])

del series

plt.figure(dpi=90, figsize=(24, 4))
plt.subplot(131)
plt.plot(time, z_mean, label='z', marker='.')
plt.xlabel("timestep")
plt.ylabel("$z \, \mathrm{[\mu m]}$")
plt.plot(time, p_z_mean*1e18, label='p_z', marker='.', c='r')
plt.hlines(0, 1.2e4, 9e4, color='black')
plt.legend()
#plt.savefig(fname="temp", facecolor="white", dpi=200)

plt.subplot(132)
print(np.mean(F_z_mean)*1e3, "mN")
plt.hlines(0, 1.2e4, 9e4, color='black')
plt.errorbar(time, F_z_mean, F_z_std, label='F_z', marker='.', capsize=3)
plt.xlabel("timestep")
plt.ylabel("$F_z \, \mathrm{[N]}$")
#plt.ylim([-7e-6,7e-6])

plt.subplot(133)
plt.hlines(0, 1.2e4, 9e4, color='black')
plt.plot(time, F_y_mean, label='F_y', marker='.')
plt.plot(time, F_y_peak, label='F_y_peak', marker='.')
plt.xlabel("timestep")
plt.ylabel("$F_y \, \mathrm{[N]}$")
plt.legend()
plt.show()


In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(20000, 30000+1, 2000)

particle = series.iterations[70000].particles["b_all"]
weight, x_pos, y_pos_end, z_pos, particleId_end = particlePMD(particle, mom=False, Id=True)
print(particleId_end, len(particleId_end))
for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    
    y_pos_end_sort = np.zeros(len(y_pos_end))
    weight, x_pos, y_pos, z_pos, particleId = particlePMD(particle, mom=False, Id=True)
    print(particleId, len(particleId))
#    Z = [x for _,x in np.sort(zip(particleId,particleId_end))]
                
    statistic, bin_y, bin_z, rest = stats.binned_statistic_2d(y_pos, z_pos, (F_x_L, F_y_L, F_z_L), statistic='mean', bins=60)
    y, z = binned_to_grid(bin_y, bin_z)
    col = np.sqrt(statistic[0].T**2 + statistic[1].T**2 + statistic[2].T**2)*1e3
    
    plt.figure(dpi=60, figsize=(24, 6))
    plt.subplot(131)
    plt.hist2d(y_pos, z_pos, bins=200,  weights=weight, cmap='hot_r', norm=LogNorm())
    plt.colorbar(label='particle density')
    plt.streamplot(y, z, statistic[1].T, statistic[2].T, density=2, color=col, cmap='Greens')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))
    
    plt.subplot(132)
    plt.streamplot(y, z, statistic[1].T, statistic[2].T, density=3, color=col, cmap='magma_r')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-z Lorentz-Force, timstep: {:1d}".format(t))
    
    statistic, bin_x, bin_z, rest = stats.binned_statistic_2d(x_pos, z_pos, (F_x_L, F_y_L, F_z_L), statistic='mean', bins=60)
    x, z = binned_to_grid(bin_x, bin_z)
    col = np.sqrt(statistic[0].T**2 + statistic[1].T**2 + statistic[2].T**2)*1e3
    
    plt.subplot(133)
    plt.streamplot(x, z, statistic[0].T, statistic[2].T, density=3, color=col, cmap='magma_r')
    plt.colorbar(label='$F_{Lorentz} \, \mathrm{[mN]}$')
    plt.xlabel("$x \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("x-z Lorentz-Force, timstep: {:1d}".format(t))

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

## y-Lorentz-Force

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(20000, 30000+1, 2000)
bins=201
dQ = constants.e * 1e12

y_hist = np.zeros((len(time), bins))
E_hist = np.zeros((len(time), bins)) 

# get y_pos at last timestep to show how particles changed position
particle = series.iterations[time[-1]].particles["b_all"]
weight_end, x_pos, y_pos_end, z_pos, x_mom_end, y_mom_end, z_mom_end, particleId_end = particlePMD(particle, Id=True)
E_end = mom_to_E(x_mom_end, y_mom_end, z_mom_end, weight_end)

for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom, x_probeE, y_probeE, z_probeE, x_probeB, y_probeB, z_probeB, particleId = particlePMD(particle, probeE=True, probeB=True, Id=True)

    # size of viewing window
    y_min, y_max, z_min, z_max = window(t)
    
    dy = (y_max - y_min) / bins
    dz = (z_max - z_min) / bins
    
    # converting momenta to velocity
    x_vel = np.sqrt((x_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))
    y_vel = np.sqrt((y_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))
    z_vel = np.sqrt((z_mom / (constants.m_e*weight))**2 / (1 + (x_mom**2 + y_mom**2 +z_mom**2) / (constants.m_e*weight)**2 / constants.c**2))

    F_y_B = z_vel*x_probeB - x_vel*z_probeB
    F_z_B = x_vel*y_probeB - y_vel*x_probeB

    F_y_L = -constants.e*weight * (y_probeE + F_y_B) *1e3
    F_z_L = -constants.e*weight * (z_probeE + F_z_B) *1e3

    print("Min of F_y_L:", np.min(F_y_L), "mN")
    print("Min of F_z_L:", np.min(F_z_L), "mN")

    hist_Fy = stats.binned_statistic_2d(y_pos, z_pos, (F_y_L), statistic='mean', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]
    hist_Fy *= 1/dy/dz
    
    E = mom_to_E(x_mom, y_mom, z_mom, weight)
    
    hist_Fz = stats.binned_statistic_2d(y_pos, z_pos, (F_z_L), statistic='mean', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]
    hist_Fz *= 1/dy/dz
    
    hist_E = stats.binned_statistic_2d(y_pos, z_pos, (E), statistic='mean', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]
    hist_E *= 1/dy/dz
    
    mask = np.isin(particleId, particleId_end)
    ind = np.argsort(particleId[mask])
    ind_end = np.argsort(particleId_end)
    hist_y_pos = stats.binned_statistic_2d((y_pos[mask])[ind], (z_pos[mask])[ind], (y_pos_end[ind_end]), statistic='min', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]
    hist_y_pos *= 1/dy/dz

    y_hist[i] = hist_E[:, 100]

    hist_E_end = stats.binned_statistic_2d((y_pos[mask])[ind], (z_pos[mask])[ind], (E_end[ind_end]), statistic='min', bins=bins, range=[[y_min, y_max], [z_min, z_max]])[0]
    hist_E_end *= 1/dy/dz
    E_hist[i] = hist_E_end[:, 100]
    

    plt.figure(dpi=60, figsize=(24, 12))
    
    plt.subplot(231)
    plt.hist2d(y_pos, z_pos, bins=200,  weights=weight, cmap='hot_r', norm=LogNorm(vmin=5e4, vmax=3e6), range=[[y_min, y_max], [z_min, z_max]])
    plt.colorbar(label='particle density')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("bunch-density, timstep: {:1d}".format(t))
    
    plt.subplot(232)
    plt.imshow(hist_Fy.T, cmap='seismic', extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none', norm=SymLogNorm(linthresh=5e-2, vmin=-1.5, vmax=1.5))
    plt.colorbar(label='$F_{Lorentz, y}/y/z \, \mathrm{[mN/\mu m^2]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-Lorentz-Force, timstep: {:1d}".format(t))
    
    plt.subplot(233)
    plt.imshow(hist_Fz.T, cmap='seismic', extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none', norm=SymLogNorm(linthresh=5e-2, vmin=-1.5, vmax=1.5))
    plt.colorbar(label='$F_{Lorentz, z}/y/z \, \mathrm{[mN/\mu m^2]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("z-Lorentz-Force, timstep: {:1d}".format(t))

    plt.subplot(234)
    plt.imshow(hist_y_pos.T, cmap='plasma', extent=(y_min, y_max, z_min, z_max), aspect='auto', vmin=np.min(y_pos_end)/dy/dz, vmax=np.max(y_pos_end)/dy/dz, origin="lower", interpolation='none')
    plt.colorbar(label='$y_{90.000}/y/z \, \mathrm{[\mu m/\mu m^2]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("y-Postion at step 90.000, timstep: {:1d}".format(t))    

    plt.subplot(235)
    plt.imshow(hist_E.T, cmap='jet', vmin=0, vmax=300, extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
    plt.colorbar(label='$E/y/z \, \mathrm{[meV/\mu m^2]}$')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("kinetic Energy, timstep: {:1d}".format(t))

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)

del series

# doch Einheiten zurück ändern?

y-position arguments over time

In [ ]:
plt.figure(dpi=60, figsize=(16, 6))
plt.subplot(121)
plt.imshow(y_hist, cmap="plasma", origin="lower", aspect='auto', vmin=np.min(y_pos_end), vmax=np.max(y_pos_end), extent=(windowwidth * 0.52, windowwidth * 0.72, time[0]-1e3, time[-1]+1e3), interpolation='none')
plt.colorbar(label="$y_{90 000} \, \mathrm{[\mu m]}$")
plt.xlabel("$y-c\cdot t \, \mathrm{[\mu m]}$")
plt.ylabel("$timestep$")

plt.subplot(122)
plt.imshow(E_hist, cmap="jet", origin="lower", aspect='auto', vmin=np.min(0), vmax=np.max(300), extent=(windowwidth * 0.52, windowwidth * 0.72, time[0]-1e3, time[-1]+1e3), interpolation='none')
plt.colorbar(label="$E_{120 000} \, \mathrm{[MeV]}$")
plt.xlabel("$y-c\cdot t \, \mathrm{[\mu m]}$")
plt.ylabel("$timestep$")
plt.show()


## Test Multidimensional color

In [ ]:
series = io.Series(path, io.Access.read_only)
time = range(16000, 90000+1, 2000)

vmin = 5e4
vmax = 3e6

map_h = np.linspace(2/3, 0, 200)
map_s = (np.log10(np.linspace(vmin, vmax, 200)) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin))
map_hs = np.meshgrid(map_h, map_s)
map_v = np.ones((200, 200))

maphsv = np.dstack((map_hs[0], map_hs[1], map_v))
maprgb = np.transpose(hsv_to_rgb(maphsv), axes=(1,0,2))


for i, t in enumerate(time):
    particle = series.iterations[t].particles["b_all"]
    
    weight, x_pos, y_pos, z_pos, x_mom, y_mom, z_mom = particlePMD(particle)
    
    E = mom_to_E(x_mom, y_mom, z_mom)
    
    plt.figure(dpi=60, figsize=(16, 6))

    statistic, bin_y, bin_z, rest = stats.binned_statistic_2d(y_pos, z_pos, (E), statistic='mean', bins=200)
    statistic2, bin_y2, bin_z2, rest2 = stats.binned_statistic_2d(y_pos, z_pos, (weight), statistic='sum', bins=200)
    
    hue  = (-(np.clip(statistic, 0, 300) / 300)+1)*2/3
    saturation = (np.log10(np.clip(statistic2, vmin, vmax)) - np.log10(vmin)) / (np.log10(vmax) - np.log10(vmin))
    saturation[statistic2==0] = 0
#    print(saturation[100])
#    print(statistic2[100])
    value = np.ones_like(statistic)
    
    hsv = np.dstack((hue, saturation, value))
    rgb = np.transpose(hsv_to_rgb(hsv), axes=(1,0,2))
    

    plt.subplot(121)
    plt.imshow(rgb, extent=(bin_y[0], bin_y[-1], bin_z[0], bin_z[-1]), aspect='auto', origin="lower", interpolation='none')
    plt.xlabel("$y \, \mathrm{[\mu m]}$")
    plt.ylabel("$z \, \mathrm{[\mu m]}$")
    plt.title("kinetic Energy, timstep: {:1d}".format(t))
        
    plt.subplot(122)
    plt.imshow(maprgb, origin="lower", aspect='auto', extent=(5e4, 3e6, 0, 300))
    plt.xlabel("$density$")
    plt.ylabel("$Energy \, \mathrm{[MeV]}$")
    plt.xscale('log')

    plt.show()
#    plt.savefig(fname="temp", facecolor="white", dpi=300)
del series

# energy muss geklippt werden bei 300 MeV
# Transponieren
# Sättigung logarithmisch

In [ ]:
map_h = np.linspace(0, 1, 21)
map_s = np.linspace(0, 1, 21)
map_hs = np.meshgrid(map_h, map_s)
print(map_hs)
plt.imshow(map_hs[1]*map_hs[0], cmap='plasma', norm=LogNorm(vmin=0.01, vmax=1))
plt.colorbar()
plt.show()

In [ ]:
div_max = 50
weight = np.ones(int(5e4), dtype=np.int32) * 5e4

E1_gauss = np.random.normal(loc=245, scale=5, size=int(3e4))
div1_gauss = np.random.normal(loc=0, scale=8, size=int(3e4))

E2_gauss = np.random.normal(loc=208, scale=20, size=int(2e4))
div2_gauss = np.random.normal(loc=0, scale=20, size=int(2e4))

E_gauss = np.append(E1_gauss, [E2_gauss])
div_gauss = np.append(div1_gauss, [div2_gauss])
h, x_edge, y_edge, r = plt.hist2d(E_gauss, div_gauss, weights=weight, bins=[1000, 201], cmap="hot_r", range=[[0, 400], [-div_max, div_max]], norm=LogNorm(vmin=1e4))
plt.colorbar()

plt.show()

x = y_edge[:-1]
xwidth = x[1] - x[0]
x += xwidth/2
y = np.sum(h, axis=0)

p0 = [8e7, 0, 5]
parameters, covariance = curve_fit(Gauss, x, y, p0)
fit_a = parameters[0]
fit_b = parameters[1]
fit_c = parameters[2]
fit_y = Gauss(x, fit_a, fit_b, fit_c)

plt.bar(x, y)
plt.plot(x, fit_y, label='fit', c='orange')

plt.show()
#E3_gauss = Gauss3(xx, 160, 20, 0.15e6)
#div3_gauss = Gauss3(yy, 0, 30, 1)
